In [ ]:
pip install emoji

In [ ]:
import os
import sys
import re
import xml.etree.ElementTree as ET
import emoji
from html import unescape
from emoji import UNICODE_EMOJI

def listXML(path):
    #return a list with the xml name files
    # r=root, d=directories, f = files
    xml_list = []
    for r, d, f in os.walk(path):
        for file in f:
            if '.xml' in file:
                xml_list.append(file)
    return xml_list


def concatenateTweets(xml_list,source_path):
    id_authors = [] # ids of authors
    x_train = [] #each element of the list is composed of all the tweets concatenated which have been written by an author 
    for p in xml_list:
        h, tail = os.path.split(p) #return ("",'id.xml')
        tail = tail.split(".")
        author = tail[0]
        id_authors.append(author)
        tree = ET.parse(source_path+str(p))
        root = tree.getroot()
        for child in root:
            xi = []
            for ch in child:
                xi.append(ch.text)
            content = ' '.join(xi)
            x_train.append(content)
    return id_authors, x_train


def hateLabels(path,spa_id_authors):
    text = open(path,"r")
    data = text.read().split("\n")
    labels = {}
    test = [] #class labels for authors
    for line in data:
        l = line.split(":::")
        if len(l) > 1:
            labels[l[0]] = l[1]
    for author in spa_id_authors:
        test.append(labels[author])
    return test

def emoji2word(text):
    return emoji.demojize(text)

def cleaning_v1(tweet_lista):#english tweets
    cleaned_feed_v1=[]
    for feed in tweet_lista:
        feed = feed.lower()
        feed = re.sub('[^0-9a-z #@]', "", feed)
        feed = re.sub('[\n]', " ", feed)
        cleaned_feed_v1.append(feed)
    return cleaned_feed_v1

def cleanTweets(tweet_list):#spanish tweets
    clean_list = []
    for t in tweet_list:
        t = t.lower()
        t = re.sub(r'\d+', "xnumber", t)
        t = re.sub('#user#', '', t)
        t = re.sub('#url#','xurl',t)
        t = re.sub('#rt#', '', t)
        t = re.sub('#hashtag#','',t)
        t = emoji2word(t) 
        t = re.sub('[,.\'\"\‘\’\”\“]', '', t)
        t = re.sub(r'(?<![?!:;/])([:\'\";.,?()/!])(?= )','',t) 
        t = re.sub(r'([a-z\'-\’]+)', r'\1 ', t)
        t = re.sub('[\n]', ' ', t)
        t = ' '.join(t.split())
        clean_list.append(t)
    return clean_list

# **Load Data**

In [ ]:
source_path = '/content/drive/MyDrive/PAN-21/dataset'
#spanish data
spa_xml_list = listXML(source_path+'/es/')
spa_id_authors, spa_x_train = concatenateTweets(spa_xml_list,source_path+'/es/') 
spa_y_train = hateLabels(source_path+'/es/truth.txt',spa_id_authors)# we obtain labels of each autor
spa_clean_x_train = cleanTweets(spa_x_train) 


We shuffle the dataset.

In [ ]:
import random
x_train = spa_clean_x_train.copy()
y_train = spa_y_train.copy()
mix = list(zip(x_train,y_train))
random.shuffle(mix)
x_train, y_train = zip(*mix)
x_train = list(x_train)
y_train = list(y_train)

# **SVM MODEL WITH TF-IDF VECTORIZATION.**
First, we use GridSearch with the objective to find the best hyperparameter configuration to SVM and Tf-idf vectorization of each set of phrases.(This process takes long time depending on the number of parameters used to test)


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
import math
param_grid = {
    'vect__ngram_range':[(1,1),(1,2),(1,3),(1,5),(1,6)],
    'vect__min_df':[0.1,0.3,0.6,0.8,2,4,6,8,12],
    'vect__analyzer':['word','char_wb'],
    'svm__C':[100,10,1],
    'svm__kernel':['linear','rbf','poly']
}
tr_x = x_train.copy()
tr_y = y_train.copy()
pipe = Pipeline([('vect', TfidfVectorizer(sublinear_tf=True, use_idf=True, smooth_idf=True)),
                ('svm', SVC())])
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(tr_x,tr_y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Once we have obtained the best configuration, we test this configuration through Cross Validation and furthermore, we finish to adjust the model by making some small variations in the hyperparameters.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
import math
corpus_particionado_x_train=[]
corpus_particionado_y_train=[]
cv = 10
num= int(len(x_train)/cv)
for i in range(cv):
    corpus_particionado_x_train.append(x_train[i*num:i*num + num])
    corpus_particionado_y_train.append(y_train[i*num:i*num + num])
print(len(corpus_particionado_x_train))
print(len(corpus_particionado_y_train))

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
import math
resultados_svm = []
for i in range(cv):
    tr_x = corpus_particionado_x_train.copy()
    te_x = tr_x[i]
    tr_y = corpus_particionado_y_train.copy()
    te_y = tr_y[i]
    del tr_x[i]
    del tr_y[i]
    tr_x = [valor for sublista in tr_x for valor in sublista]
    tr_y = [valor for sublista in tr_y for valor in sublista]
    clasificadorSVC = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,6), min_df=8, sublinear_tf=True, use_idf=True, smooth_idf=True)),
                ('svm', SVC(C=100, kernel='linear', probability=True))])
    clasificadorSVC.fit(tr_x,tr_y)
    print(clasificadorSVC.score(te_x,te_y))
    resultados_svm.append(clasificadorSVC.score(te_x,te_y))
resultados_svm
precision_media_svm = sum(resultados_svm)/len(resultados_svm)
aux = 0
print("Average accuracy with SVM is %.4f  " %(precision_media_svm))

In addition, if we want to use TF-IDF of char and word n-grams as features, we have to use FeatureUnion to combine both in one output and use it with the SVM.


In [ ]:
resultados_svm_mix = []
for i in range(cv):
    tr_x = corpus_particionado_x_train.copy()
    te_x = tr_x[i]
    tr_y = corpus_particionado_y_train.copy()
    te_y = tr_y[i]
    del tr_x[i]
    del tr_y[i]
    tr_x = [valor for sublista in tr_x for valor in sublista]
    tr_y = [valor for sublista in tr_y for valor in sublista]
    estimators =[('word',TfidfVectorizer(ngram_range=(1,6), min_df=8, sublinear_tf=True, use_idf=True, smooth_idf=True)),('char',TfidfVectorizer(ngram_range=(1,6), min_df=0.3, sublinear_tf=True, use_idf=True,analyzer = 'char_wb', smooth_idf=True))]
    combined = FeatureUnion(estimators)
    clasificadorSVCmix = Pipeline([(('vect', combined)),
                ('svm', SVC(C=100, kernel='linear', probability=True))])
    clasificadorSVCmix.fit(tr_x,tr_y)
    print(clasificadorSVCmix.score(te_x,te_y))
    resultados_svm_mix.append(clasificadorSVCmix.score(te_x,te_y))
resultados_svm_mix
precision_media_svm_mix = sum(resultados_svm_mix)/len(resultados_svm_mix)
print("Average accuracy with SVM using tfidf of words y chars combined is %.4f " %(precision_media_svm_mix))

# **RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
param_grid = {
    'vect__ngram_range':[(1,1),(1,2),(1,3)],
    'vect__min_df':[0.1,0.3,0.6,0.8,2,6,8],
    'vect__analyzer':['word'],
    'rf__n_estimators':[10,100,150,200],
    'rf__criterion':['gini'],
    'rf__min_samples_leaf':[1,2,4,6]
}
tr_rf_x = x_train.copy()
tr_rf_y = y_train.copy()
pipe = Pipeline([('vect', TfidfVectorizer(sublinear_tf=True, use_idf=True, smooth_idf=True)),
                 ('rf',RandomForestClassifier())])
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(tr_rf_x,tr_rf_y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
resultados_rf = []
for i in range(cv):
    tr_x = corpus_particionado_x_train.copy()
    te_x = tr_x[i]
    tr_y = corpus_particionado_y_train.copy()
    te_y = tr_y[i]
    del tr_x[i]
    del tr_y[i]
    tr_x = [valor for sublista in tr_x for valor in sublista]
    tr_y = [valor for sublista in tr_y for valor in sublista]
    clasificadorRF = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), min_df=8, sublinear_tf=True, use_idf=True, smooth_idf=True)),
                ('RF',RandomForestClassifier(n_estimators=100, min_samples_leaf=1, criterion='gini'))])
    clasificadorRF.fit(tr_x,tr_y)
    print(clasificadorRF.score(te_x,te_y))
    resultados_rf.append(clasificadorRF.score(te_x,te_y))
resultados_rf
precision_media_rf = sum(resultados_rf)/len(resultados_rf)
aux = 0
for res in resultados_rf:
    aux  += (res-precision_media_rf)**2 
varianza = aux/cv
ic_medio_rf = 1.96 * math.sqrt(varianza)
print("Average accuracy with rf is %.4f and its ic %.4f " %(precision_media_rf,ic_medio_rf))

# **XGBoost**

In [ ]:
import xgboost as xgb
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
import math
param_grid = {
    'vect__ngram_range':[(1,2)],
    'vect__min_df':[8],
    'vect__analyzer':['word'],
    "xgboost__eta":[.01,.1,.3,1],
    "xgboost__n_estimators":[100,200,300],
    "xgboost__max_depth": [4,5,6],
    "xgboost__subsample": [.6,.8],
    "xgboost__colsample_bytree":[.5,.7]
}
tr_x = x_train.copy()
tr_y = y_train.copy()
pipe = Pipeline([('vect', TfidfVectorizer(sublinear_tf=True, use_idf=True, smooth_idf=True)),
                ('xgboost', xgb.XGBClassifier(random_state=0))])
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(tr_x,tr_y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
import xgboost as xgb
resultados_xgb = []
for i in range(cv):
    tr_x = corpus_particionado_x_train.copy()
    te_x = tr_x[i]
    tr_y = corpus_particionado_y_train.copy()
    te_y = tr_y[i]
    del tr_x[i]
    del tr_y[i]
    tr_x = [valor for sublista in tr_x for valor in sublista]
    tr_y = [valor for sublista in tr_y for valor in sublista]
    clasificadorxgb = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), min_df=8, sublinear_tf=True, use_idf=True, smooth_idf=True)),
                ('xgboost', xgb.XGBClassifier(colsample_bytree= 0.7, eta= 0.01,max_depth=6,n_estimators=200,subsample=0.7))])
    clasificadorxgb.fit(tr_x,tr_y)
    print(clasificadorxgb.score(te_x,te_y))
    resultados_xgb.append(clasificadorxgb.score(te_x,te_y))
resultados_xgb
precision_media_xgb = sum(resultados_xgb)/len(resultados_xgb)
aux = 0
for res in resultados_xgb:
    aux  += (res-precision_media_xgb)**2 
varianza = aux/cv
ic_medio_xgb = 1.96 * math.sqrt(varianza)
print("Average accuracy with xboost is %.4f and its ic %.4f " %(precision_media_xgb,ic_medio_xgb))

# **Ensembled models**

In [ ]:
from sklearn.ensemble import VotingClassifier
resultados_ens = []
for i in range(cv):
    tr_x = corpus_particionado_x_train.copy()
    te_x = tr_x[i]
    tr_y = corpus_particionado_y_train.copy()
    te_y = tr_y[i]
    del tr_x[i]
    del tr_y[i]
    tr_x = [valor for sublista in tr_x for valor in sublista]
    tr_y = [valor for sublista in tr_y for valor in sublista]
    clasificadorRF = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), min_df=8, sublinear_tf=True, use_idf=True, smooth_idf=True)),
                ('RF',RandomForestClassifier(n_estimators=100, min_samples_leaf=1, criterion='gini'))])
    clasificadorSVC = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), min_df=8, sublinear_tf=True, use_idf=True, smooth_idf=True)),
                ('svm', SVC(C=100, kernel='linear', probability=True))])
    ensembled = VotingClassifier(estimators=[('svm', clasificadorSVC), ('RF', clasificadorRF),],voting='soft', weights=[4,1])
    clasificadorSVC.fit(tr_x,tr_y)
    clasificadorRF.fit(tr_x,tr_y)
    ensembled.fit(tr_x,tr_y)
    resultados_ens.append(ensembled.score(te_x,te_y))
resultados_ens
precision_media_ens = sum(resultados_ens)/len(resultados_ens)
aux = 0
for res in resultados_ens:
    aux  += (res-precision_media_ens)**2 
varianza = aux/cv
ic_medio_ens = 1.96 * math.sqrt(varianza)
print("Average accuracy with ensembled model is %.4f and its ic %.4f " %(precision_media_ens,ic_medio_ens))

# **Pre-trained BERT**

In [ ]:
!pip install -q tensorflow-text
!pip install -q tf-models-official
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In this case, we have created the classifier model using multi_cased BERT because the text is written in spanish. 
</br>
Due to this, we have implemented the preprocessing_layer  with "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3" and the encoder with https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3 .
</br>
We can find other pre-trained BERT models on https://tfhub.dev/google/collections/bert/1 .

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3", name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3", trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(64, activation='relu', name='dense')(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
tf.keras.utils.plot_model(classifier_model)

Next step is to convert the data to a correct format. We also split the data into 90% train and 10% test in order to prove the accuracy provided by the created model.

In [ ]:
import random
from sklearn.model_selection import train_test_split
x_train = spa_clean_x_train.copy()
y_train = spa_y_train.copy()
X_train, X_test, Y_train, Y_test = train_test_split(x_train,y_train, test_size = 0.10, random_state = 42)
for i in range(len(Y_train)):
    Y_train[i] = int(Y_train[i])
for i in range(len(Y_test)):
    Y_test[i] = int(Y_test[i])
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(10000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).shuffle(10000).batch(batch_size)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 15
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
history = classifier_model.fit(x=train_ds,
                               epochs=epochs)

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')